In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import CubicSpline
import warnings
warnings.filterwarnings("ignore")
import arviz as az
%run -i '../../utils.py'

# Read data

In [2]:
df_bac = pd.read_excel('./16S_relative_abundance.xlsx', index_col=0)

# Read metadata for inulin and control

In [3]:
df_meta = pd.read_csv('metadata.txt', sep='\t', index_col=0)
#df_meta = df_meta[['Group','Subject_ID','status','dose']]
# df_meta = df_meta[(df_meta.Group=='inulin') | (df_meta.Group=='accessible')]
# df_meta['Day'] = [0 if s=='before' else 21 for s in df_meta.status]
# df_meta['Diet'] = [1 if g=='inulin' else 0 for g in df_meta['Group']]
# df_meta = df_meta.drop('Group', axis=1)
df_meta.head()

,Sample_type,Host_Age,Subject_ID,Time_point_original,Time_point,semester,status,Group,dose,Acetate,Propionate,Butyrate,total_SCFA,Collection_Date
#SampleID,,,,,,,,,,,,,,
SRR6444565,feces,17-29 yo,U222,1,1.0,fall16,before,accessible,0,73.981150,NaN,10.548664,84.529814,2016/9/1
SRR6443884,feces,17-29 yo,U222,2,2.0,fall16,before,accessible,0,71.237839,15.792592,18.146317,105.176748,2016/9/1
SRR6444382,feces,17-29 yo,U222,3,3.0,fall16,before,accessible,0,29.743311,5.881475,10.214394,45.839180,2016/9/1
SRR6444917,feces,17-29 yo,U225,1,1.0,fall16,before,accessible,0,28.019063,13.531843,8.422327,49.973233,2016/9/1
SRR6443930,feces,17-29 yo,U225,2,2.0,fall16,before,accessible,0,17.114913,8.984568,3.395816,29.495297,2016/9/1


# Select the 20 most abundant taxa

In [5]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Bacteroides,Faecalibacterium,Lachnospiraceae,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Alistipes,Prevotella,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Parabacteroides,Bifidobacterium,Blautia,Anaerostipes,Parasutterella_dash_excrementihominis,Bacteroides_dash_plebeius_dash_DSM_dash_17135,Ruminococcus,Fusicatenibacter,gut_dash_metagenome,Roseburia,Lachnospiraceae_dash_UCG_dash_004,metagenome,Bacteroides_dash_coprocola
#SampleID,,,,,,,,,,,,,,,,,,,,
SRR6444565,0.197580,0.136827,0.024022,0.000215,0.028962,0.000215,0.037984,0.007446,0.009165,0.018580,0.006158,0.000215,0.114631,0.006623,0.003508,0.036051,0.027852,0.001539,0.009201,0.000215
SRR6443884,0.243131,0.099128,0.017050,0.000070,0.032008,0.000070,0.037064,0.006764,0.004916,0.014191,0.005230,0.000070,0.125837,0.005335,0.002789,0.034100,0.022873,0.003626,0.007985,0.000070
SRR6444382,0.214061,0.110743,0.028395,0.000167,0.031565,0.000167,0.037600,0.006981,0.007370,0.020525,0.007231,0.000167,0.115360,0.006869,0.003560,0.032734,0.029730,0.004533,0.008121,0.000167
SRR6444917,0.578520,0.063804,0.117987,0.000199,0.037393,0.000199,0.000630,0.003550,0.016358,0.014168,0.000299,0.026942,0.000199,0.003782,0.000199,0.000630,0.000199,0.000199,0.001194,0.000199
SRR6443930,0.610679,0.055143,0.050196,0.000181,0.079698,0.000181,0.000181,0.004404,0.008386,0.006154,0.000181,0.023409,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181


In [6]:
list(df_16S_topX.columns)

['Bacteroides',
 'Faecalibacterium',
 'Lachnospiraceae',
 'Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_',
 'Alistipes',
 'Prevotella',
 'Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57',
 'Parabacteroides',
 'Bifidobacterium',
 'Blautia',
 'Anaerostipes',
 'Parasutterella_dash_excrementihominis',
 'Bacteroides_dash_plebeius_dash_DSM_dash_17135',
 'Ruminococcus',
 'Fusicatenibacter',
 'gut_dash_metagenome',
 'Roseburia',
 'Lachnospiraceae_dash_UCG_dash_004',
 'metagenome',
 'Bacteroides_dash_coprocola']

# Normalize max value of 16S data to 1

In [7]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta = df_16S_topX_w_meta.groupby(['Subject_ID','status']).agg(np.mean).reset_index()
df_16S_topX_w_meta.head()

,Subject_ID,status,Day,Diet,Bacteroides,Faecalibacterium,Lachnospiraceae,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Alistipes,Prevotella,...,Anaerostipes,Parasutterella_dash_excrementihominis,Bacteroides_dash_plebeius_dash_DSM_dash_17135,Ruminococcus,Fusicatenibacter,gut_dash_metagenome,Roseburia,Lachnospiraceae_dash_UCG_dash_004,metagenome,Bacteroides_dash_coprocola
0,U042,before,0,1,0.444946,0.100588,0.080670,0.000128,0.027035,0.000128,...,0.003644,0.000128,0.000128,0.009113,0.022899,0.007670,0.025910,0.007428,0.011713,0.000128
1,U042,during,21,1,0.471585,0.037131,0.080455,0.000254,0.038842,0.000406,...,0.003687,0.000254,0.000254,0.010182,0.008189,0.016515,0.047139,0.008635,0.009576,0.000254
2,U068,before,0,1,0.045633,0.056045,0.043214,0.177269,0.000270,0.328534,...,0.000425,0.000161,0.000161,0.021873,0.004559,0.011077,0.002486,0.006962,0.011519,0.015651
3,U068,during,21,1,0.070620,0.093106,0.044719,0.120436,0.000256,0.258661,...,0.002214,0.000199,0.000153,0.008714,0.007876,0.017048,0.009354,0.004645,0.018583,0.004436
4,U075,before,0,1,0.227864,0.133095,0.049664,0.013882,0.017764,0.000785,...,0.014899,0.016442,0.000179,0.014111,0.045612,0.011263,0.018615,0.002632,0.017789,0.000179


# Calculate log-derivatives (dlog(B)/dt)

In [8]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_sub in set(df_deriv.Subject_ID):
    curr_df = df_deriv[df_deriv.Subject_ID==curr_sub].sort_values(by='Day')
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.Subject_ID==curr_sub, taxon] = ydata_d1
df_deriv.head()

,Subject_ID,status,Day,Diet,Bacteroides,Faecalibacterium,Lachnospiraceae,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Alistipes,Prevotella,...,Anaerostipes,Parasutterella_dash_excrementihominis,Bacteroides_dash_plebeius_dash_DSM_dash_17135,Ruminococcus,Fusicatenibacter,gut_dash_metagenome,Roseburia,Lachnospiraceae_dash_UCG_dash_004,metagenome,Bacteroides_dash_coprocola
0,U042,before,0,1,0.001269,-0.003022,-0.000010,0.000006,5.622348e-04,0.000013,...,0.000002,0.000006,5.970561e-06,0.000051,-0.000700,0.000421,0.001011,0.000057,-0.000102,0.000006
1,U042,during,21,1,0.001269,-0.003022,-0.000010,0.000006,5.622348e-04,0.000013,...,0.000002,0.000006,5.970561e-06,0.000051,-0.000700,0.000421,0.001011,0.000057,-0.000102,0.000006
2,U068,before,0,1,0.001190,0.001765,0.000072,-0.002706,-7.087060e-07,-0.003327,...,0.000085,0.000002,-3.810299e-07,-0.000627,0.000158,0.000284,0.000327,-0.000110,0.000336,-0.000534
3,U068,during,21,1,0.001190,0.001765,0.000072,-0.002706,-7.087060e-07,-0.003327,...,0.000085,0.000002,-3.810299e-07,-0.000627,0.000158,0.000284,0.000327,-0.000110,0.000336,-0.000534
4,U075,before,0,1,0.005197,-0.003111,-0.000158,-0.000459,6.566401e-04,-0.000024,...,0.000185,0.000248,-4.237658e-06,0.000450,-0.000997,0.000204,-0.000491,0.000112,-0.000448,-0.000004


# Construct regression matrix

In [10]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([ 0.89588099,  0.89588099,  0.83846764, ..., -2.43909054,
       -0.01323803, -0.01323803])

In [11]:
len(Ymat)

3560

In [12]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.00000000e+00, 1.00000000e+00, 4.44946301e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 4.71585345e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 4.56331153e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.46741789e-03, 6.74048698e-03, 6.98077419e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.20216398e-02, 3.61930501e-03, 2.97519882e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.50391700e-02, 4.41138219e-03, 7.73246735e-04]])

# Write stan program

## write data

In [15]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"inulin_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [16]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] inulin_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*inulin_response_%s+'%(c1,c1) # inulin response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [9]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [10]:
fit = az.from_cmdstan(["output_%d_inulin.csv"%(i) for i in np.arange(1,4)])

In [11]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['inulin_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-0.36,0.66,0.150,0.584667,False
1,basal_growth_rate,Faecalibacterium,-0.92,0.09,-0.415,-1.623619,False
2,basal_growth_rate,Lachnospiraceae,-0.72,0.31,-0.205,-0.761524,False
3,basal_growth_rate,Trichuris_dash_trichiura_dash__leftroundbracke...,-0.48,0.52,0.020,0.067067,False
4,basal_growth_rate,Alistipes,-0.35,0.66,0.155,0.600150,False


In [12]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20_inulin.xlsx')

# Plot

In [14]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20_inulin.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
8,basal_growth_rate,Bifidobacterium,0.03,1.06,0.545,2.116647,True
21,inulin_response,Faecalibacterium,0.26,0.85,0.555,3.799384,True
23,inulin_response,Trichuris_dash_trichiura_dash__leftroundbracke...,0.13,0.70,0.415,2.871528,True
26,inulin_response,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,-0.64,-0.08,-0.360,-2.507589,True
28,inulin_response,Bifidobacterium,0.48,1.05,0.765,5.313565,True
30,inulin_response,Anaerostipes,0.13,0.71,0.420,2.886729,True
42,pairwise_interaction,"('Bacteroides', 'Lachnospiraceae')",-4.73,-1.15,-2.940,-3.240980,True
43,pairwise_interaction,"('Bacteroides', 'Trichuris_dash_trichiura_dash...",-3.53,-0.67,-2.100,-2.880639,True
48,pairwise_interaction,"('Bacteroides', 'Bifidobacterium')",-4.12,-0.62,-2.370,-2.633815,True
61,pairwise_interaction,"('Faecalibacterium', 'Faecalibacterium')",0.59,3.96,2.275,2.663886,True
